In [1]:
import pandas as pd

df=pd.read_csv("../Data/アニメゲーム.csv")
df
# 열 Q, K1, K2, K3, K4의 내용을 합쳐서 하나의 열로 합치기
df['QK'] = df['Q'] +","+ df['K1'] +","+  df['K2'] + ","+ df['K3'] +","+  df['K4']
df=df[['QK','A']]
#열 이름을 text, prediction으로 고치기
df.columns=['text','prediction']
df


,text,prediction
0,"1997年に公開された映画『シト新生』は、何というアニメの劇場版作ロ\r\nロロ ?,機動警...",機動警察パトレイバー
1,"アニメ『ゲゲゲの鬼太郎』で、鬼太郎に助けて欲しい時に手紙を入れるボストの名前は何ポスト ?,...",悪霊ポスト
2,その展開がよく「起承転結」というーー >ニス混面ビリロリテリ\r\n四字熟語で例えられる漫...,4コマ漫画
3,1990年に発売された、「ファミリーコンピュータ」の後継機であるゲーム機は「何ファミコン」 ...,ミラクルファミコン
4,"石川チカの漫画 !交番PB』に登場するコンビ、仙波と武田の職業は何 ?,郵便局員,消防士,韻...",韻察到
...,...,...
2813,"の だゴクEi【ひ28較=レン舞台にした作品はどれ ?,イヴの有眠り,カリフォルニア物語,海...",海街diary
2814,"アニメ 「ルパン三世』に登場するルパンの仲間、五工門の名字は何 ?,次元,峰,勾形,石川",石川
2815,"小林よしのりの漫画『おぼっちゃまくん』で、おぼっちゃまくんは「歩く何」と呼ばれていた ?,売...",身代金
2816,"次のうち、魔法使いの少女を主人公とする漫画はどれ2?,アルスラーン戦記,将棋の渡辺くん,ふら...",ふらいんぐうつういっち


In [2]:
import numpy as np
from datasets import load_dataset


train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.99 * len(df)), int(0.999 * len(df))]
)

print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

Source News : バーネットの小説『小公女」を原作としたアニメのタイトルは 『小公女何』 ?,小公女カーラ,小公女オーラ,小公女ヒーラ,小公女セーラ
Summarization : 小公女セーラ
Training Data Size : 2789
Validation Data Size : 26
Testing Data Size : 3


In [3]:
import torch
from transformers import T5Tokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence


def make_dataset(data, tokenizer, device):
    source = tokenizer(
        text=data.text.tolist(),
        padding="max_length",
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )

    target = tokenizer(
        text=data.prediction.tolist(),
        padding="max_length", 
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )
    
    source_ids = source["input_ids"].squeeze().to(device)
    source_mask = source["attention_mask"].squeeze().to(device)
    target_ids = target["input_ids"].squeeze().to(device)
    target_mask = target["attention_mask"].squeeze().to(device)
    return TensorDataset(source_ids, source_mask, target_ids, target_mask)

def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 20
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path="t5-small"
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(next(iter(train_dataloader)))
print(tokenizer.convert_ids_to_tokens(21603))
print(tokenizer.convert_ids_to_tokens(10))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[tensor([[    3,     2,  4482,  ...,     0,     0,     0],
        [    3,     2,   254,  ...,     0,     0,     0],
        [    3,     2,     3,  ...,     0,     0,     0],
        ...,
        [    3,     2,     3,  ...,     0,     0,     0],
        [    3,     2, 19479,  ...,     0,     0,     0],
        [    3,     2,    58,  ...,     0,     0,     0]], device='cuda:0'), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), tensor([[3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0],
        ...,
        [3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0]], device='cuda:0'), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0]

In [4]:
from torch import optim
from transformers import T5ForConditionalGeneration


model = T5ForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="t5-small",
).to(device)
model.load_state_dict(torch.load(f"../Model/T5_アニメゲーム.pt"))
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [5]:
import numpy as np
from torch import nn
from tqdm import tqdm  # tqdm 임포트 추가


def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for source_ids, source_mask, target_ids, target_mask in tqdm(dataloader, desc="Training"):  # tqdm 추가
        decoder_input_ids = target_ids[:, :-1].contiguous()
        labels = target_ids[:, 1:].clone().detach()
        labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
        )

        loss = outputs.loss
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss


def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        val_loss = 0.0

        for source_ids, source_mask, target_ids, target_mask in tqdm(dataloader, desc="Evaluation"):  # tqdm 추가
            decoder_input_ids = target_ids[:, :-1].contiguous()
            labels = target_ids[:, 1:].clone().detach()
            labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

            outputs = model(
                input_ids=source_ids,
                attention_mask=source_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels,
            )

            loss = outputs.loss
            val_loss += loss

    val_loss = val_loss / len(dataloader)
    return val_loss


best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f}")

    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(), "../Model/T5_アニメゲーム.pt")
        print("Saved the model weights")


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 30.81it/s]


Epoch 1: Train Loss: 1.1642 Val Loss: 0.1004
Saved the model weights


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 30.92it/s]


Epoch 2: Train Loss: 0.3821 Val Loss: 0.1016
Saved the model weights


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 29.91it/s]


...

Epoch 15: Train Loss: 0.1105 Val Loss: 0.0482


Evaluation: 100%|██████████| 4/4 [00:00<00:00, 28.80it/s]


Epoch 16: Train Loss: 0.0885 Val Loss: 0.0501
Saved the model weights


Training:  10%|█         | 35/349 [00:06<00:57,  5.48it/s]


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    for source_ids, source_mask, target_ids, target_mask in train_dataloader:
        generated_ids = model.generate(
            input_ids=source_ids,
            attention_mask=source_mask,
            max_length=128,
            num_beams=3,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
        )

        for generated, target in zip(generated_ids, target_ids):
            pred = tokenizer.decode(
                generated, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            actual = tokenizer.decode(
                target, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            print("Generated Headline Text:", pred) 
            print("Actual Headline Text   :", actual) 
        break

Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: ,
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : !!
Generated Headline Text: 
Actual Headline Text   : 


In [6]:
from transformers import pipeline


summarizer = pipeline(
    task="summarization",
    model=model,
    tokenizer=tokenizer,
    max_length=128,
    device="cpu"
)

for index in range(0,10):
    news_text = df.text.iloc[index]
    summarization = df.prediction.iloc[index]
    predicted_summarization = summarizer(news_text)[0]["summary_text"]
    print(f"정답 요약문 : {summarization}")
    print(f"모델 요약문 : {predicted_summarization}\n")

Your max_length is set to 128, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 128, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


정답 요약문 : 機動警察パトレイバー
모델 요약문 : , . - : ?  ( ) & !



Your max_length is set to 128, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


정답 요약문 : 悪霊ポスト
모델 요약문 : ?,, . ? ? . ! ?? !



Your max_length is set to 128, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


...

정답 요약문 : ニンテンドー3DS
모델 요약문 : PSVita,3DS,XboxOne, 4DS,xboxOne . 5DS,DSVITa .



Your max_length is set to 128, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


정답 요약문 : ウルトラマン
모델 요약문 : , ?  . ,.  ( ( ! &



Your max_length is set to 128, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


정답 요약문 : モンキー・D・ルフィ
모델 요약문 : jEI  .D -D D iDD DIDi

정답 요약문 : 天空の城ラピュタ
모델 요약문 : ?, , ( . ! &  ( ..

